In [ ]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [2]:
#path = DATA_DIR/'sample_gtfs.zip'
#path = DATA_DIR/'other_feeds'/'gtfs.zip'
path = DATA_DIR/'cairns_gtfs.zip'

print(gt.list_gtfs(path))

feed = gt.read_gtfs(path, dist_units='km')


            file_name  file_size
0  calendar_dates.txt        387
1          routes.txt       1478
2           trips.txt     143081
3           stops.txt      26183
4          agency.txt        199
5      stop_times.txt    2561019
6          shapes.txt     864694
7        calendar.txt        337


In [5]:
trip_stats = feed.compute_trip_stats()
dates = feed.get_dates()

In [123]:
ts = feed.compute_route_time_series(trip_stats, dates[:1], freq='6H')
rids = ['110-423', '111-423', '112-423'] 
cols = ['num_trips', 'num_trip_ends', 'num_trip_starts']

# #ts.xs(rid, level='route_id', axis=1)
ts.loc[:, (cols, rids)]

indicator           num_trip_ends                 num_trip_starts          \
route_id                  110-423 111-423 112-423         110-423 111-423   
2014-05-26 00:00:00             0       0       0               1       0   
2014-05-26 06:00:00            19      18       4              22      22   
2014-05-26 12:00:00            24      24       6              24      24   
2014-05-26 18:00:00            15      15       5              12      12   

indicator                   num_trips                  
route_id            112-423   110-423 111-423 112-423  
2014-05-26 00:00:00       0         1       0       0  
2014-05-26 06:00:00       5        23      22       5  
2014-05-26 12:00:00       6        28      28       7  
2014-05-26 18:00:00       4        15      15       5

In [124]:
dts = gt.downsample(ts, freq='D')
dts.loc[:, (cols, rids)]

indicator  num_trip_ends                 num_trip_starts                  \
route_id         110-423 111-423 112-423         110-423 111-423 112-423   
2014-05-26            58      57      15              59      58      15   

indicator  num_trips                  
route_id     110-423 111-423 112-423  
2014-05-26        58      57      15

In [117]:
feed.compute_route_stats(trip_stats, dates[:1])

,route_id,route_short_name,route_type,num_trips,is_loop,is_bidirectional,start_time,end_time,max_headway,min_headway,mean_headway,peak_num_trips,peak_start_time,peak_end_time,service_distance,service_duration,service_speed,mean_trip_distance,mean_trip_duration,date
0,110-423,110,3,59,0,1,05:50:00,24:02:00,35.0,23.0,29.955556,5,18:13:00,18:20:00,1894.223063,57.350000,33.029173,32.105476,0.972034,20140526
1,111-423,111,3,58,0,1,06:02:00,24:36:00,67.0,25.0,30.954545,5,07:57:00,08:05:00,2002.677623,59.350000,33.743515,34.528925,1.023276,20140526
2,112-423,112,3,15,1,0,07:55:00,22:31:00,60.0,60.0,60.000000,1,07:55:00,08:31:00,317.421880,9.000000,35.269098,21.161459,0.600000,20140526
3,113-423,113,3,6,0,1,06:05:00,18:42:00,60.0,60.0,60.000000,2,06:35:00,06:45:00,147.601565,3.933333,37.525822,24.600261,0.655556,20140526
4,120-423,120,3,32,0,1,05:34:00,22:23:00,60.0,60.0,60.000000,2,07:00:00,07:23:00,899.055146,26.633333,33.756764,28.095473,0.832292,20140526
5,120N-423,120N,3,2,0,0,22:00:00,23:51:00,NaN,NaN,NaN,1,22:00:00,22:51:00,81.004241,1.700000,47.649553,40.502120,0.850000,20140526
6,121-423,121,3,34,0,1,06:28:00,22:00:00,60.0,30.0,52.800000,3,07:46:00,07:48:00,586.058987,18.133333,32.319429,17.237029,0.533333,20140526
7,122-423,122,3,33,0,1,06:16:00,21:30:00,60.0,30.0,54.000000,2,07:16:00,07:30:00,541.875382,15.400000,35.186713,16.420466,0.466667,20140526
8,123-423,123,3,60,0,1,06:14:00,24:15:00,50.0,10.0,29.565217,3,07:10:00,07:31:00,1127.869577,40.466667,27.871571,18.797826,0.674444,20140526
9,130-423,130,3,33,0,1,06:04:00,23:01:00,60.0,60.0,60.000000,2,06:30:00,06:35:00,361.049921,17.050000,21.175948,10.940907,0.516667,20140526


In [ ]:
n = 6
rng = pd.date_range('1/1/2001', periods=n, freq='H')
split_directions = True

if split_directions:    
    mi = [(dt, i) for dt in rng for i in range(2)]
    f = pd.DataFrame(np.random.randint(7, size=(len(mi), 2)), 
      index = pd.MultiIndex.from_tuples(mi, names=['time', 'direction']),
      columns =['t', 'e']) 
    f = f.unstack('direction') 
else:
    f = pd.DataFrame(np.random.randint(7, size=(n, 2)), 
      index=rng, columns =['t', 'e']) 

f

In [ ]:
from pandas.tseries.resample import TimeGrouper


def my_agg(group):
    return group['t'].iloc[-1] + group['e'].iloc[:-1].sum()

f.groupby(TimeGrouper('3H')).apply(my_agg)

In [ ]:
feed.compute_stop_stats(['20140601', '20140603'])

In [ ]:
trip_stats = feed.compute_trip_stats()
feed = feed.append_dist_to_stop_times(trip_stats)

# Load screen line
with (DATA_DIR/'cairns_screen_line.geojson').open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])



In [ ]:
f = feed.compute_screen_line_counts(line, dates[:7])
f[f['crossing_time'] < '06:00:00']

In [ ]:
feed.compute_feed_time_series(ts, dates[-2:], freq='12H')

In [ ]:
sd = False
#%time p1 = proto1(feed, ts, dates[:14], split_directions=sd)
%time p2 = proto2(feed, [dates[0], '20010101'], split_directions=sd, freq='12H')
p2

In [ ]:
feed.compute_feed_stats(ts, dates[0])

In [ ]:
feed.describe()

In [ ]:
feed.summarize()